# 🚀 AI Banner Creator - Google Colab (FIXED)
**Free GPU • Unlimited • Zero Cost • Robust**

Tạo banner quảng cáo với AI Stable Diffusion
- ✅ Removed problematic transformers & rembg
- ✅ Fixed all dependency conflicts
- ✅ Uses local storage instead of Google Drive
- ✅ Error handling built-in

In [ ]:
# Cell 1: GPU + PyTorch Installation
!nvidia-smi

print('\n📦 Installing PyTorch with CUDA...')
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print('📦 Installing AI libraries...')
!pip install -q diffusers pillow opencv-python accelerate omegaconf einops

print('\n✅ GPU & Dependencies setup complete!')

In [ ]:
# Cell 2: Setup Folders
import os
os.makedirs('/content/BannerCreator/input', exist_ok=True)
os.makedirs('/content/BannerCreator/output', exist_ok=True)

print('✅ Folders created:')
print('   📁 /content/BannerCreator/input')
print('   📁 /content/BannerCreator/output')

In [ ]:
# Cell 3: Download Stable Diffusion 2.1
import torch
from diffusers import StableDiffusionPipeline

print('📥 Downloading Stable Diffusion 2.1 (7GB)...')
print('⏱️ First time: 3-5 minutes')
print('⏳ Please wait...\n')

try:
    sd_pipeline = StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        torch_dtype=torch.float16,
        safety_checker=None
    )
    sd_pipeline = sd_pipeline.to("cuda")
    print('✅ Stable Diffusion 2.1 ready!')
    print(f'📊 Device: {"CUDA (GPU)" if torch.cuda.is_available() else "CPU"}')
except Exception as e:
    print(f'❌ Error: {e}')
    print('💡 Restart kernel and try again')

In [ ]:
# Cell 4: Banner Creator Class
import io
import requests
from datetime import datetime
from PIL import Image, ImageDraw, ImageFont

class CoLabBannerCreator:
    def __init__(self):
        self.base_path = "/content/BannerCreator"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
    def download_sample_image(self):
        """Download sample product image"""
        sample_path = f"{self.base_path}/input/sample_product.jpg"
        
        if not os.path.exists(sample_path):
            print('📥 Downloading sample product...')
            try:
                url = "https://images.unsplash.com/photo-1505740420928-5e560c06d30e?w=500"
                img = Image.open(io.BytesIO(requests.get(url).content))
                img.save(sample_path)
                print(f'✅ Saved to: {sample_path}')
            except:
                print('⚠️ Cannot download. Using placeholder.')
                img = Image.new('RGB', (400, 400), color=(100, 150, 200))
                img.save(sample_path)
        
        return sample_path
    
    def create_simple_background(self, width=1000, height=600):
        """Create simple gradient background"""
        bg = Image.new('RGB', (width, height))
        pixels = bg.load()
        
        for y in range(height):
            r = int(30 + (y / height) * 80)
            g = int(100 + (y / height) * 120)
            b = int(150 + (y / height) * 70)
            
            for x in range(width):
                pixels[x, y] = (r, g, b)
        
        return bg
    
    def generate_background(self, prompt, width=1000, height=600):
        """Generate background using Stable Diffusion"""
        print(f'🎨 Generating background...')
        
        with torch.no_grad():
            image = sd_pipeline(
                prompt=prompt,
                height=height,
                width=width,
                num_inference_steps=20,
                guidance_scale=7.5
            ).images[0]
        
        return image
    
    def create_banner(self, product_path, title, subtitle, ai_prompt=None):
        """Create complete banner"""
        print(f'\n{"="*55}')
        print(f'📊 Creating Banner: {title}')
        print(f'{"="*55}')
        
        # Load product image
        product_img = Image.open(product_path)
        product_img.thumbnail((350, 350), Image.Resampling.LANCZOS)
        
        # Generate or create background
        if ai_prompt:
            try:
                bg_img = self.generate_background(ai_prompt, width=1000, height=600)
            except Exception as e:
                print(f'⚠️ AI failed: {e}. Using gradient.')
                bg_img = self.create_simple_background()
        else:
            bg_img = self.create_simple_background()
        
        # Ensure correct mode
        if product_img.mode != 'RGBA':
            product_img = product_img.convert('RGBA')
        bg_img = bg_img.convert('RGBA')
        
        # Composite product in center
        x = (bg_img.width - product_img.width) // 2
        y = (bg_img.height - product_img.height) // 2
        bg_img.paste(product_img, (x, y), product_img)
        
        # Convert to RGB for saving
        bg_img = bg_img.convert('RGB')
        
        # Add text
        draw = ImageDraw.Draw(bg_img)
        try:
            font_title = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 52)
            font_subtitle = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 36)
        except:
            font_title = font_subtitle = ImageFont.load_default()
        
        # Add shadow for better visibility
        shadow_color = (0, 0, 0)
        text_color = (255, 255, 255)
        
        draw.text((52, 52), title, font=font_title, fill=shadow_color)
        draw.text((50, 50), title, font=font_title, fill=text_color)
        
        draw.text((52, 122), subtitle, font=font_subtitle, fill=shadow_color)
        draw.text((50, 120), subtitle, font=font_subtitle, fill=text_color)
        
        # Save banner
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"{self.base_path}/output/banner_{timestamp}.png"
        bg_img.save(output_path)
        
        print(f'✅ Banner saved: {output_path}')
        return output_path

# Initialize
creator = CoLabBannerCreator()
print('✅ Banner Creator initialized!')

In [ ]:
# Cell 5: Download Sample Image
sample_path = creator.download_sample_image()

In [ ]:
# Cell 6: Create First Banner (with AI)
banner1 = creator.create_banner(
    product_path=sample_path,
    title="🎯 Premium Product",
    subtitle="Limited Offer - 50% OFF",
    ai_prompt="Modern minimalist background with blue gradient, professional design, clean"
)

print(f'\n✅ Banner created: {banner1}')

In [ ]:
# Cell 7: Create Multiple Banners (Batch)
banners_config = [
    {"title": "🌞 Summer Sale", "subtitle": "70% OFF", "prompt": "bright sunny beach, golden sand, azure water"},
    {"title": "🎁 Black Friday", "subtitle": "Mega Deals", "prompt": "dark luxury, red accents, premium elegant"},
    {"title": "🚀 New Launch", "subtitle": "Be First", "prompt": "futuristic tech, neon lights, modern"}
]

print(f'📊 Creating {len(banners_config)} banners...\n')

for i, cfg in enumerate(banners_config, 1):
    print(f'[{i}/{len(banners_config)}]', end=' ')
    try:
        creator.create_banner(
            product_path=sample_path,
            title=cfg["title"],
            subtitle=cfg["subtitle"],
            ai_prompt=cfg["prompt"]
        )
    except Exception as e:
        print(f'❌ {e}')

print('\n✅ All banners created!')

In [ ]:
# Cell 8: Display Results
import glob
from IPython.display import Image as IPImage, display

output_dir = f"{creator.base_path}/output/"
banners = sorted(glob.glob(output_dir + "*.png"))

print(f'📸 Total banners created: {len(banners)}\n')

# Show last 3 banners
for banner_path in banners[-3:]:
    print(f'📄 {os.path.basename(banner_path)}')
    display(IPImage(banner_path))

print(f'\n💾 All files saved in: {output_dir}')

In [ ]:
# Cell 9 (Optional): Create Custom Banner
# Change these values:
custom_banner = creator.create_banner(
    product_path=sample_path,
    title="✨ Your Custom Title",
    subtitle="Your custom subtitle here",
    ai_prompt="Your background description here"
)

In [ ]:
# Cell 10 (Optional): Download Banner to Computer
from google.colab import files
import glob

# Get latest banner
latest_banner = sorted(glob.glob(f"{creator.base_path}/output/*.png"))[-1]

print(f'📥 Downloading: {os.path.basename(latest_banner)}')
files.download(latest_banner)

## ✅ Hoàn thành!

### 📊 Hiệu năng:
- **GPU T4**: 30-60 giây/banner
- **CPU**: 2-5 phút/banner
- **Chi phí**: FREE

### 🚀 Bước tiếp theo:
1. Cell #1: Setup GPU (1 min)
2. Cell #2: Create folders (5 sec)
3. Cell #3: Download SD 2.1 (3-5 min)
4. Cell #4-8: Create banners (30-60 sec each)

### 💡 Tips:
- Lần đầu: download models, mất 5-10 phút
- Lần sau: tức thì (models cached)
- GPU T4 có 12 giờ → ~400 banners/ngày

Enjoy! 🎉